# Linked Data con Python

Usaremos:

* gastrodon: Toolkit to display, analyze, and visualize data and documents based on RDF graphs and the SPARQL query language using Pandas, Jupyter, and other Python ecosystem tools. https://github.com/paulhoule/gastrodon
* sys
* pandas 

<div class="alert alert-warning" role="alert" style="margin: 10px">
Para instalar la librería:<br>
jovyan@9778f76bfaea:~$ /opt/conda/bin/pip install gastrodon
</div>

## Cargamos el entorno

In [ ]:
import sys
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from IPython.display import display, HTML
pd.options.display.width=120
pd.options.display.max_colwidth=100

Definimos los prefijos que de los "namespace" a utilizar

In [ ]:
prefixes=inline("""
    @prefix : <http://dbpedia.org/resource/> .
    @prefix dbo: <http://dbpedia.org/ontology/> .
    @prefix dbp: <http://dbpedia.org/property/> .
    @prefix foaf: <http://xmlns.com/foaf/0.1/> .
""").graph

Y el endpoint sobre el que vamos a hacer las consultas. En este caso, dbpedia.

In [ ]:
endpoint=RemoteEndpoint(
    "http://dbpedia.org/sparql/"
    ,default_graph="http://dbpedia.org"
    ,prefixes=prefixes
    ,base_uri="http://dbpedia.org/resource/"
)

## Tripletas en DBpedia

Con una consulta simple, podemos contar el número de tripletas

In [ ]:
count=endpoint.select("""
    SELECT (COUNT(*) AS ?count) { ?s ?p ?o .}
""").at[0,"count"]
count

## Número de predicados

Para la siguiente consulta, hacemos una lista de predicados comunes. El número de respuestas en el endpoint de DBPEDIA está limitado a 10000.

Each predicate is a relationship between a topic and either another topic or a literal value.  For instance,  the **rdf:type** predicate links a topic to another topic representing a class that the first topic is an instance,  for instance:

```
<Alan_Alda> rdf:type on:Person .
```

**rdfs:label**,  on the other hand,  links topics to literal values,  such as

```
<Alan_Alda> rdfs:label 
                "Alan Alda"@en,
                "アラン・アルダ"@ja . 
```

Strings in RDF (like the one above) are unusual compared to other computer languages because they can contain language tags,  a particularly helpful feature for multilingual databases such as DBpedia.

In [ ]:
predicates=endpoint.select("""
    SELECT ?p (COUNT(*) AS ?count) { ?s ?p ?o .} GROUP BY ?p ORDER BY DESC(?count)
""")
predicates

Podemos enseñar solo los N primeros resultados

In [ ]:
predicates.head(30)

O los últimos

In [ ]:
predicates.tail()

## Visualización de resultados

**dbo:Image** Algunos resultados de dbpedia son links a imagenes, que gracias a Jupyter, podemos visualizarlos

In [ ]:
endpoint.select("""
    SELECT (COUNT(*) AS ?count) { SELECT DISTINCT ?type { ?s a ?type .} }
""")

In [ ]:
types=endpoint.select("""
    SELECT ?type (COUNT(*) AS ?count) { ?s a ?type .} GROUP BY ?type ORDER BY DESC(?count)
""")
types[types.index.str.startswith('dbo:')]

In [ ]:
pics = endpoint.select("""
    SELECT ?that { 
        ?that a dbo:Image
    } LIMIT 10
""")
pics

In [ ]:
HTML('<img src="{0}">'.format(pics.at[9,'that']))

## Ejercicio 1

Muestra la imagen de un escritor/a vivo

In [ ]:
escritores = endpoint.select("""
SELECT ?nombre ?foto
WHERE{
    ?autor a dbo:Writer .
    ?autor foaf:name ?nombre .
    ?autor dbo:birthDate ?fechaNac .
    ?autor foaf:depiction ?foto
    FILTER NOT EXISTS { ?autor dbo:deathDate ?death. }
} 
LIMIT 100



""")
escritores

In [ ]:
HTML('<img src="{0}">'.format(escritores.at[99,'foto']))

Los datos se pueden recoger en distintos tipos de estructuras para mostrarlos en gráficos. Veamos un ejemplo de un diagrama de barras.

In [ ]:
places = endpoint.select("""
SELECT ?city ?population
WHERE{
    ?city a dbo:PopulatedPlace .
    ?city dbo:populationTotal ?population
    FILTER (?population > 1000000)
} 
ORDER BY ASC(?population)
LIMIT 100
""")

In [ ]:
places.head(10)

In [ ]:
places.head(10).hist()
places.head(10)['city']

# Ejercicios de consultas

<div class="alert alert-warning" role="alert" style="margin: 10px">
La librería a veces tiene problemas con las fechas. Si es así, colocal la consulta en el endpoint directamente para comprobar que si el problema está en la consulta o en el paquete.
</div>

1. Obtener la masa de 10 estrellas

2. Constelación a la que pertenece cada estrella 

3. Distancia a la que está la estrella Kepler-19 

4. Estrellas pertenecientes a la constelación Centaurus 

5. Obtener la población total de Londres 

6. Gente que nació en Londres antes de 1900 

7. Nombre y fechas de nacimiento y de muerte de toda la gente nacida en Londres entre 1900 y 1950. 

8. Nombre y fecha de nacimiento de los actores nacidos en Londres después de 1930 que siguen vivos.

9. Fotos de 10 de los actores nacidos en Londres después de 1930 que siguen vivos.

10 . Histograma de las 10 peliculas más largas